In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
import findspark
findspark.init()

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!gzip -dc fhvhv_tripdata_2021-01.csv.gz

In [4]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [4]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [8]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [5]:
import pandas as pd

In [6]:
df_pandas = pd.read_csv('head.csv')

In [7]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [8]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

Integer - 4 bytes
Long - 8 bytes

In [5]:
from pyspark.sql import types

In [6]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [11]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [16]:
df = df.repartition(24)

In [17]:
df.write.parquet('fhvhv/2021/01/')

In [7]:
df = spark.read.parquet('fhvhv/2021/01/')

In [8]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [9]:
from pyspark.sql import functions as F

In [10]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-03 04:15:54|2021-01-03 04:31:22|          82|         129|   NULL|
|           HV0005|              B02510|2021-01-01 06:17:49|2021-01-01 06:41:07|          85|          76|   NULL|
|           HV0003|              B02836|2021-01-01 13:21:04|2021-01-01 13:36:54|         254|         153|   NULL|
|           HV0003|              B02878|2021-01-03 15:49:14|2021-01-03 16:00:21|          94|         174|   NULL|
|           HV0003|              B02878|2021-01-01 03:08:49|2021-01-01 03:27:15|         256|          37|   NULL|
|           HV0003|              B02835|2021-01-02 08:51:20|2021-01-02 09:23:34|

In [11]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [12]:
crazy_stuff('B02884')

's/b44'

In [13]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [14]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-03|  2021-01-03|          82|         129|
|  e/9ce| 2021-01-01|  2021-01-01|          85|          76|
|  e/b14| 2021-01-01|  2021-01-01|         254|         153|
|  e/b3e| 2021-01-03|  2021-01-03|          94|         174|
|  e/b3e| 2021-01-01|  2021-01-01|         256|          37|
|  s/b13| 2021-01-02|  2021-01-02|          35|         205|
|  e/b47| 2021-01-01|  2021-01-01|          74|          94|
|  e/9ce| 2021-01-01|  2021-01-01|          63|          14|
|  e/b38| 2021-01-01|  2021-01-01|         137|          25|
|  s/b44| 2021-01-02|  2021-01-02|         126|          83|
|  s/acd| 2021-01-03|  2021-01-03|         213|         248|
|  e/9ce| 2021-01-02|  2021-01-02|          74|         224|
|  e/9ce| 2021-01-03|  2021-01-03|          94|         169|
|  e/b3e| 2021-01-01|  2

In [15]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [16]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,
